In [26]:
import sys
sys.path.append('./privatizers')
sys.path.append('./dataProviders')
sys.path.append('./adapters')

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import fetch_covtype
from adapters import generalAdapter as adapter
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

dp = fetch_covtype()

X = dp.data
Y = dp.target

In [27]:
# ## Size adjustment
# desired_size = 10000
# min_test_size = 0.3
# test_size = 0.3

# total_size = len(X)

# if (desired_size < min_test_size * total_size):
#     test_size = 1-(desired_size / total_size);

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = test_size)

# ## Dataset
# data_set = {
#     'samples': X_train,
#     'labels': y_train
# }

In [ ]:
## Multiple Dataset sizes
desired_sizes = [10, 100, 1000, 10000]
data_sets = []

min_test_size = 0.2

total_size = len(X)

for desired_size in desired_sizes:
    test_size = 0.2
    if (desired_size < (1-min_test_size) * total_size):
        test_size = 1-(desired_size / total_size);

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = test_size)

    ## Dataset
    single_data_set = {
        'samples': X_train,
        'labels': y_train
    }
    data_sets.append(single_data_set)

In [28]:
for data_set in data_sets:
    ## SVM without privatization
    clf = MLPClassifier(solver='adam', hidden_layer_sizes=(100, 50), max_iter=1000)
    clf.fit(data_set['samples'], data_set['labels'])  

    y_pred = clf.predict(X_test)
    print(str(accuracy_score(y_test,y_pred)).replace('.', ','))

    ## With cross validation
    # scores = cross_val_score(clf, X, Y, cv=5)
    # for v in scores:
    #     print(str(v).replace('.', ','))

0,4772001288939637


In [ ]:
## Privatization: Laplace privatizer
from privatizers import laplacePrivatizer

for data_set in data_sets:
    ad = adapter.GeneralAdapter(7, 1)

    dataInput = ad.fromRaw(data_set['samples'])
    dataTarget = ad.fromRaw(data_set['labels'])

    privatizer = laplacePrivatizer.LaplacePrivatizer(1.0)

    inputSensitivity = privatizer.getSensitivityList(dataInput)
    targetSensitivity = 7

    epsilon = [1.0, 1.5, 2.0, 2.5, 3.0]

    privatizer = laplacePrivatizer.LaplacePrivatizer(1.0)
    privateData = privatizer.privatize(dataInput, sensitivityList = inputSensitivity)
    for i in epsilon:
        privatizer = laplacePrivatizer.LaplacePrivatizer(i)
        privateData = privatizer.privatize(dataInput, sensitivityList = inputSensitivity)

        privateTargetsFloat = privatizer.privatize(dataTarget, sensitivityList = targetSensitivity)
        privateTargets = ad.toDiscreteValue(privateTargetsFloat)

        clf = MLPClassifier(solver='adam', hidden_layer_sizes=(100, 50), max_iter=1000)
        clf.fit(privateData, privateTargets)

        y_pred = clf.predict(X_test)
        print(str(accuracy_score(y_test,y_pred)).replace('.', ','))

0,5944691179870125
0,6531211253003439
0,5806795654031789
0,6536377519211505
0,6552103983804193
0,54591847456796
0,6568215729266635
0,6761276470547029
0,6185491723466408
0,6326959853733372
0,4845624960596275
0,4226303475233445
0,4452253192577389
0,4888863981842763
0,3992227834091052
0,4072734023102842
